In [1]:
# %pip install pm4py
# %pip install pyemd

import os
import time
import pickle
import pandas as pd
import numpy as np
import subprocess
import multiprocessing as mp
import argparse
import pathlib
from tqdm import tqdm
import warnings
import xml.parsers.expat
import warnings
import random
from pm4py.objects.log.importer.xes import importer as xes_importer
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.petri_net.obj import PetriNet, Marking
from pm4py.objects.petri_net.importer import importer as pnml_importer
from pm4py.algo.simulation.playout.petri_net import algorithm as simulator
from pm4py.statistics.variants.log import get as variants_module
from pm4py.algo.evaluation.earth_mover_distance import algorithm as emd_evaluator
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.util import dataframe_utils

warnings.filterwarnings("ignore", category=UserWarning, module='pm4py.utils')
warnings.filterwarnings("ignore", category=UserWarning, module='pm4py.objects.stochastic_petri')
warnings.filterwarnings("ignore", category=UserWarning, module='pm4py.algo.simulation.montecarlo')

#os.chdir(r'/users/sima/Desktop/stochastic/EMD-GROUP')


/Users/emilio-imt/Library/Python/3.9/lib/python/site-packages/pm4py/algo/evaluation/earth_mover_distance/__init__.py:23: UserWarning: The earth_mover_distance package will be removed in a future release.
  warnings.warn("The earth_mover_distance package will be removed in a future release.")


# **The main estimation**

In [8]:
import warnings
import random
from pm4py.objects.log.importer.xes import importer as xes_importer


train_logfile =r'bubble2_train.xes'
log = pm4py.read_xes(train_logfile)


log['case:concept:name'] = log['case:concept:name'].astype(str)
log['concept:name']      = log['concept:name'].astype(str)
# log['time:timestamp']    = pd.to_datetime(log['time:timestamp'], utc=True)
log['time:timestamp'] = pd.to_datetime(log['time:timestamp'], format="%Y-%m-%dT%H:%M:%S", utc=True)

test_logfile_path1 = r'bubble2_test.csv'
full_log1 =  pd.read_csv(test_logfile_path1)

log = pm4py.convert_to_event_log(log)
language_model = variants_module.get_language(log)
# xes_exporter.apply(language_model, 'language_model_of_train.xes')
print('language of model is produced')
def perform_analysis(num_groups,language_model, test_log,seed=None,limit=1000):
    test_log = test_log.copy()

    test_log['case:concept:name'] = test_log['case:concept:name'].astype(str)
    test_log['concept:name']      = test_log['concept:name'].astype(str)
    # test_log['time:timestamp']    = pd.to_datetime(test_log['time:timestamp'], utc=True)
    #format="%Y-%m-%d %H:%M:%S"
    test_log['time:timestamp'] = pd.to_datetime(test_log['time:timestamp'], utc=True,format="ISO8601")

    # Get unique case IDs and divide them into num_groups
    case_ids = test_log['case:concept:name'].unique()
    # Shuffle case_ids with a seed
    if seed is not None:
        random.seed(seed)
        np.random.shuffle(case_ids)


    grouped_case_ids = np.array_split(case_ids, num_groups)

    emd_results = []
    
    for idx,group_case_ids in enumerate(tqdm(grouped_case_ids)):
        
        group_df = test_log[test_log['case:concept:name'].isin(group_case_ids)]

        if not group_df.empty:
            traditional_log = pm4py.convert_to_event_log(group_df)
            case_language = variants_module.get_language(traditional_log)
            emd_value = emd_evaluator.apply(language_model, case_language)
            emd_results.append(emd_value)
        if(idx>=limit):
            break

    # Convert the list of results to a DataFrame
    emd_df = pd.DataFrame({'EMD Value': emd_results})

    return emd_df



def main(language_model, full_log1, num_groups):
    emd_estimation_start_time_group = time.time()
    emd_df = perform_analysis(num_groups, language_model, full_log1)
    average_emd = emd_df['EMD Value'].mean()

    emd_estimation_time_group = time.time() - emd_estimation_start_time_group
    return emd_df,average_emd, emd_estimation_time_group

# nruns = [5000]
group_counts = [1, 2 ,10 ,50 ,200,500, 1000,2000,5000 ,10000]
#group_counts = [1, 2 ,10 ,50 ,200,500, 1000]
#group_counts = [10000]


seed = 117

# Directory path for the tracking DataFrame
tracking_df_directory = 'EMD-GROUP-BUBBLE2'
# Check if directory exists, and create it if it doesn't
if not os.path.exists(tracking_df_directory):
    os.makedirs(tracking_df_directory)

tracking_df_group = pd.DataFrame(columns=['Num Groups', 'EMD Estimation Time', 'EMD CSV File'])
tracking_df_group = pd.DataFrame(columns=['Num Groups', 'Average EMD', 'EMD Estimation Time', 'EMD CSV File'])


for num_groups in group_counts:
    print(f'EMD analysis for num_groups= {num_groups} is started')

    emd_df,average_emd , emd_estimation_time_group= main(language_model, full_log1, num_groups)
    print('Emd evaluated')
    csv_file_path = f'{tracking_df_directory}/EMD_SOLO_nGroup{num_groups}.csv'
    emd_df.to_csv(csv_file_path, index=False)
    print(f'EMD results for num_groups= {num_groups} saved to {csv_file_path}')

    new_row = {
        'Num Groups': num_groups,
        'Average EMD': average_emd,
        'EMD Estimation Time': emd_estimation_time_group,
        'EMD CSV File': csv_file_path,
    }

    tracking_csv_file_path = os.path.join(tracking_df_directory, "EMD_SOLO_tracking_groups.csv")

    tracking_df_group = pd.concat([tracking_df_group, pd.DataFrame([new_row])], ignore_index=True)
    tracking_df_group.to_csv(tracking_csv_file_path, index=False)
    print(f'Tracking DataFrame updated for num_groups= {num_groups}')

    print("-------------------------------------------------------------------------------------------------")


print('Analysis finished')



parsing log, completed traces ::   0%|          | 0/10000 [00:00<?, ?it/s]

language of model is produced
EMD analysis for num_groups= 1 is started


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.62s/it]


Emd evaluated
EMD results for num_groups= 1 saved to EMD-GROUP-BUBBLE2/EMD_SOLO_nGroup1.csv
Tracking DataFrame updated for num_groups= 1
-------------------------------------------------------------------------------------------------
EMD analysis for num_groups= 2 is started


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  3.00s/it]


...

Emd evaluated
EMD results for num_groups= 2000 saved to EMD-GROUP-BUBBLE2/EMD_SOLO_nGroup2000.csv
Tracking DataFrame updated for num_groups= 2000
-------------------------------------------------------------------------------------------------
EMD analysis for num_groups= 5000 is started


 20%|████████████████████████▌                                                                                                  | 1000/5000 [23:54<1:35:36,  1.43s/it]


Emd evaluated
EMD results for num_groups= 5000 saved to EMD-GROUP-BUBBLE2/EMD_SOLO_nGroup5000.csv
Tracking DataFrame updated for num_groups= 5000
-------------------------------------------------------------------------------------------------
EMD analysis for num_groups= 10000 is started


 10%|████████████▏                                                                                                             | 1000/10000 [23:28<3:31:12,  1.41s/it]

Emd evaluated
EMD results for num_groups= 10000 saved to EMD-GROUP-BUBBLE2/EMD_SOLO_nGroup10000.csv
Tracking DataFrame updated for num_groups= 10000
-------------------------------------------------------------------------------------------------
Analysis finished


# **Statistics**

In [30]:
group_counts = [1, 2 ,10 ,50 ,200,500, 1000,2000,5000 ,10000]
#group_counts = [1, 2 ,10 ,50 ,200,500]
res=[]
res_solo=pd.read_csv(f"EMD-GROUP-BUBBLE2/EMD_SOLO_tracking_groups.csv")
runtime=round(res_solo["EMD Estimation Time"],2)
for gc in reversed(group_counts):
    idx=group_counts.index(gc)
    df_res=pd.read_csv(f"EMD-GROUP-BUBBLE2/EMD_SOLO_nGroup{gc}.csv")
    df_res_sel=pd.read_csv(f"EMD-GROUP-SEL/EMD_SOLO_nGroup{gc}.csv")
    thresh=pd.read_csv(f"EMD-GROUP-BUBBLE2/EMD_SOLO_nGroup{group_counts[max(idx,0)]}.csv")["EMD Value"].mean()
    tp=round(df_res[df_res["EMD Value"]<=thresh].shape[0]*100/df_res.shape[0],2)
    fn=round(100-tp,2)
    fp=round(df_res_sel[df_res_sel["EMD Value"]<=thresh].shape[0]*100/df_res_sel.shape[0],2)
    tn=round(100-fp,2)
    res.append([gc,tp,fn,fp,tn,round(thresh,4)])
    #print("gc=",gc,"tp=",tp,"fn=",fn,"fp=",fp,"tn=",tn,round(thresh,4))
    #print("gc=",gc,"tp=",tp,"fn=",fn,round(thresh,4))
    print("%d & %.4f & %.2f & 0 & %.2f&-&-&-&-\\\\"%(10000//gc,round(thresh,4),runtime[idx],fn))
    #10000 &0.0093&10.50&0&00.00&-&-&-&-\\

1 & 0.2215 & 1408.85 & 0 & 49.45&-&-&-&-\\
2 & 0.1895 & 1434.86 & 0 & 46.85&-&-&-&-\\
5 & 0.1522 & 1435.05 & 0 & 48.35&-&-&-&-\\
10 & 0.1276 & 1502.51 & 0 & 45.80&-&-&-&-\\
20 & 0.1050 & 769.92 & 0 & 49.40&-&-&-&-\\
50 & 0.0781 & 328.13 & 0 & 51.00&-&-&-&-\\
200 & 0.0443 & 117.19 & 0 & 50.00&-&-&-&-\\
1000 & 0.0190 & 27.28 & 0 & 50.00&-&-&-&-\\
5000 & 0.0060 & 6.70 & 0 & 50.00&-&-&-&-\\
10000 & 0.0000 & 4.32 & 0 & 0.00&-&-&-&-\\


[45963.07,
 23045.58,
 9219.71,
 4674.0,
 2426.64,
 1072.01,
 356.66,
 76.58,
 17.5,
 10.5]